<a href="https://colab.research.google.com/github/SuzukiRyotaro1998/stock_AI/blob/main/feature_engineering/genetic_argo_lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb
import os

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# N-Lags model check
# !pip install yfinance
import yfinance as yf
# 他ファイルのインポート・リロード------------------------
import imp
import joblib
from sklearn.preprocessing import PolynomialFeatures

!pip install deap
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import random



In [ ]:

def data_process(data,save_path):
    count = 0
    preprocess_data = pd.DataFrame()

    # # 作業ディレクトリの移動と確認
    os.chdir('/content/drive/MyDrive/machine_learning/data_preprocess/')
    # 他ファイルのインポート・リロード
    import data_process
    imp.reload(data_process)
    for  file in data['symbol'] :
        count = count +1
        print(count)
        print(file)
#         try:
        df = yf.download(file, start='2014-01-01')
        df = data_process.create_data(df)
        df = df['2016-01-06':'2020-12-31']
        preprocess_data = preprocess_data.append(df)
        print(df)

#         except: 
#             print('errror')
#             continue
    # drop row contains NaN
    preprocess_data.dropna(inplace=True)
    preprocess_data.to_csv(save_path)
    print('saved')

    return preprocess_data


# 全時系列データの呼び出し
model_stock = 'nasdaq100'
path = '/content/drive/MyDrive/stock_data/stock_code/{}.csv'.format(model_stock)
data = pd.read_csv(path)
print(data)

save_path = '/content/drive/MyDrive/machine_learning/light_gbm/feature_engineering/preprocss_data.csv'
#データの前処理　※一度やったら飛ばす
# df = data_process(data,save_path)

df = pd.read_csv(save_path, index_col=0)
X = df.copy()
# X から NaN を含む列を削除する
X = X.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
y = X.pop('return')



# 多項式特徴量を追加
poly = PolynomialFeatures(2)
poly.fit(X)
X_sc  = pd.DataFrame(poly.transform(X), columns=poly.get_feature_names(input_features=X.columns))
print(X_sc)

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X_sc, y, discrete_features)
mi_scores[::3]  # show a few features with their MI scores

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")


plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)



mi_scores = mi_scores.sort_values(ascending=True)
feature_list = mi_scores.index.values.tolist()
point_list = mi_scores.values.tolist()

selected_feature_list = feature_list[0:100]
selected_point_list = point_list[0:100]
print('------------------------------------------------')
print('特徴量')
print(selected_feature_list)
print(selected_point_list)
print('------------------------------------------------')

In [ ]:
# yはそのまま
X = X[selected_feature_list]
print(X)


#目的関数の定義。#必ずreturnの後に,をつける
#特徴量が30を超えてはいけないので、100を超えた場合、価値を0にします。
# この操作をするかどうか今後決める

def evalIndividual(individual):
    """ 個体評価のための関数
        Prameter:
        individual: 要素が0or1のリスト  長さは入力変数の数と同じ
        Return:
        n_features: 使った特徴量の数
        score: モデルの精度
    """
    n_features = sum(individual)

    if n_features == 0:
        return -9999,
    
    if n_features > 30:
        return -9999,

    X_poly_sc = X_poly.iloc[:, [bool(val) for val in individual]]
    
    X_, X_test, y_, y_test = train_test_split(X_poly_sc, y, test_size=0.15, random_state=11)
    X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.3, random_state=11)
    # train用・valid用のデータセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val) 

    params = {'metric': 'rmse',
              'max_depth' : 20,
              'num_leaves': 23, # 決定木の複雑度を調整（初期値31）'num_leaves': 23, # 決定木の複雑度を調整（初期値31）
              'boosting_type': 'gbdt', # 勾配ブースティング
              'objective': 'regression', # 目的 : 回帰  
              'num_leaves': 23, # 決定木の複雑度を調整（初期値31）
              'min_data_in_leaf': 15, # データの最小数（初期値20） 
              "learning_rate": 0.025400643228660445,
              'gamma':0.20387750446308275, 
              }

    model = lgb.train(params,
                    lgb_train,
                    valid_sets=lgb_eval,
                    num_boost_round=10000,
                    early_stopping_rounds=100,
                    verbose_eval=50
                    )
    
    # テストデータの予測
    y_pred = model.predict(X_test)
    # 真値と予測値の表示
    df_pred = pd.DataFrame({'CRIM':y_test,'CRIM_pred':y_pred})
    display(df_pred)

    # 散布図を描画(真値 vs 予測値)
    plt.plot(y_test, y_test, color = 'red', label = 'x=y') # 直線y = x (真値と予測値が同じ場合は直線状に点がプロットされる)
    plt.scatter(y_test, y_pred) # 散布図のプロット
    plt.xlabel('y') # x軸ラベル
    plt.ylabel('y_test') # y軸ラベル
    plt.title('y vs y_pred') # グラフタイトル
    
    # モデル評価
    # rmse : 平均二乗誤差の平方根
    mse = mean_squared_error(y_test, y_pred) # MSE(平均二乗誤差)の算出
    rmse = np.sqrt(mse) # RSME = √MSEの算出
    print('RMSE :',rmse)

    # r2 : 決定係数
    r2 = r2_score(y_test,y_pred)
    print('R2 :',r2)



    return 1/rmse,


#最大化問題として設定
creator.create( "FitnessMax", base.Fitness, weights=(1.0,) )
#個体の定義（list型と指定しただけで、中身の遺伝子は後で入れる）
creator.create("Individual", list, fitness = creator.FitnessMax )


#各種関数の設定を行います
#交叉、選択、突然変異などには、DEAPのToolbox内にある関数を利用
toolbox = base.Toolbox()
#random.uniformの別名をattribute関数として設定。各個体の遺伝子の中身を決める関数
toolbox.register("attr_bool", random.randint, 0, 1)
#individualという関数を設定。それぞれの個体に含まれる遺伝子をattributeにより決めるよ、ということ。
toolbox.register("individual", tools.initRepeat, creator.Individual, 
                 toolbox.attr_bool, X_poly.shape[1])
#集団の個体数を設定するための関数を準備
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#評価したい関数の設定（目的関数のこと）
toolbox.register("evaluate", evalIndividual)
#交叉関数の設定。ブレンド交叉という手法を採用
# toolbox.register("mate", tools.cxBlend,alpha=0.2)
toolbox.register("mate", tools.cxTwoPoint)
#突然変異関数の設定。indpbは各遺伝子が突然変異を起こす確率。muとsigmaは変異の平均と標準偏差
# toolbox.register("mutate", tools.mutGaussian, mu=[0.0, 0.0], sigma=[20.0, 20.0], indpb=0.2)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
#トーナメント方式で次世代に子を残す親を選択（tornsizeは各トーナメントに参加する個体の数）
toolbox.register("select", tools.selNSGA2)
# toolbox.register("select", tools.selTournament, tournsize=5)

"""遺伝的アルゴリズム設定
50世代まで
1世代の個体数300
次世代に引き継ぐ個体数100
交叉確率0.7
突然変異確率0.1
"""
#乱数の固定
random.seed(128)
#何世代まで行うか
NGEN =50
#集団の個体数
MU =300
LAMBDA = 100
#交叉確率
CXPB = 0.7
#個体が突然変異を起こす確率
MUTPB = 0.1


#集団は300個体という情報の設定
pop = toolbox.population(n=MU)


#集団内の個体それぞれの適応度（目的関数の値）を計算
# for individual in pop:
#     individual.fitness.values = toolbox.evaluate(individual)
    
    
hof = tools.ParetoFront()
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

pop, log = algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB,
                                     MUTPB, NGEN, stats, halloffame=hof)






# joblib.dump(model, '/Users/suzukiryotaro/Desktop/python_stock/machine_learning/ansamble/lightgbm/train/lightgbm0706.joblib') 





